<a href="https://colab.research.google.com/github/swathikuppala/Atoms/blob/main/embeddings_through_encoder_decoder_framework_(GCN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=9d7785a12b724b2e7c99cd4a7a851b041361d005a95b7da55646a067287f167a
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.0/627.0 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.1/504.1 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.3/732.3 kB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 14.0 MB/s eta 0:00:00


In [ ]:
import torch
from torch_geometric.data import Data
import torch_geometric.transforms as T
from torch_geometric.data import InMemoryDataset
from torch_geometric.loader import DataLoader
import numpy as np

In [ ]:
# mounting the Google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
headDir = "drive/MyDrive/dataset/"
#headDir = "../"

In [ ]:
path = headDir + "andrea-juslin-dataGt100(2).npy"
data = np.load(path, allow_pickle=True)
data = data[None][0]
dataXorig = []
dataY = []
dataPe = []
for x, y, z in zip(data['x'], data['y'], data['z']):
  if y > 1: continue # We only want 0 and 1 labels for this activity
  dataXorig.append(x)
  dataY.append(y)
  dataPe.append(z['pe'])

In [ ]:
featLen = 30 # we are restricting to first thirty nearest neighbours
dataAtoms = torch.tensor(np.array([x[0][:featLen+1] for x in dataXorig]), dtype=torch.float)
dataX = torch.tensor(np.array([x[1][:featLen+1] for x in dataXorig]).reshape(len(dataAtoms),featLen+1,1), dtype=torch.float)
dataY = torch.tensor(dataY, dtype=torch.long)
#dataX = dataAtoms

In [ ]:
dataXAll = torch.concat((dataX, dataAtoms), axis=2)

In [ ]:
dataXAll.shape, dataX.shape, dataY.shape, dataAtoms.shape

(torch.Size([3582, 31, 4]),
 torch.Size([3582, 31, 1]),
 torch.Size([3582]),
 torch.Size([3582, 31, 3]))

In [ ]:
posData = []
for dt, y in zip(dataAtoms, dataY):
  data = Data(x= dt, pos=dt, y = y)#, pre_transform=T.RadiusGraph(r=4.0), transform=T.Distance())
  data.validate(raise_on_error=True)
  posData.append(data)

In [ ]:
distData = []
for dt, x, y in zip(dataAtoms, dataX, dataY):
  data = Data(x= x, pos=dt, y = y)#, pre_transform=T.RadiusGraph(r=4.0), transform=T.Distance())
  data.validate(raise_on_error=True)
  distData.append(data)

In [ ]:
allData = []
for dt, x, y in zip(dataAtoms, dataXAll, dataY):
  data = Data(x= x, pos=dt, y = y)#, pre_transform=T.RadiusGraph(r=4.0), transform=T.Distance())
  data.validate(raise_on_error=True)
  allData.append(data)

In [ ]:
allData = posData

In [ ]:
class MyDataset(InMemoryDataset):
    def __init__(self, root, data_list, transform=None, pre_transform=None, pre_filter=None):
        self.data_list = data_list
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['mydata']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        # Download to `self.raw_dir`.
        pass

    def process(self):
        # Read data into huge `Data` list.
        data_list = self.data_list

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
dir(data)
data.edge_weight

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
#!rm -rf ./data/processed/
dataset = MyDataset("./data",allData, transform=T.Compose([T.RadiusGraph(r=1.0), T.Distance(),T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
]))
#dataset = MyDataset("./data",allData, pre_transform=T.Compose([T.RadiusGraph(r=2.0), T.Distance()]))


Processing...
Done!


In [ ]:
train_data, val_data, test_data = dataset[0]

In [ ]:
dataset.data

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The data of the dataset is already cached, so any modifications to `data` will not be reflected when accessing its elements. Clearing the cache now by removing all elements in `dataset._data_list`. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[111042, 3], y=[3582], pos=[111042, 3], edge_index=[2, 667386], edge_attr=[667386, 1])

In [ ]:
import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


model = Net(dataset.num_node_features, 128, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()



In [ ]:
train_data.x, train_data.edge_index, train_data.num_nodes

(tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 4.9373e-01, -5.0243e-01,  4.9416e-01],
         [ 4.9772e-01,  5.0008e-01,  5.0076e-01],
         [ 4.9486e-01, -4.9807e-01, -5.0623e-01],
         [-5.0338e-01,  4.9722e-01,  5.0005e-01],
         [ 4.9590e-01,  5.0237e-01, -5.0386e-01],
         [-5.0824e-01,  4.9111e-01, -5.0395e-01],
         [-5.0035e-01, -5.0338e-01,  5.0045e-01],
         [-5.0382e-01, -4.9987e-01, -5.0158e-01],
         [-7.6340e-03,  9.9569e-01,  2.3790e-04],
         [-4.5994e-03, -2.4343e-03,  9.9725e-01],
         [ 9.9891e-01, -3.8582e-03,  1.6801e-03],
         [-1.0004e+00, -5.1671e-03,  2.2675e-03],
         [-1.4050e-03, -6.5336e-03, -1.0009e+00],
         [-3.7660e-03, -1.0029e+00, -2.3496e-03],
         [-6.9563e-03,  9.8963e-01, -1.0010e+00],
         [ 9.9740e-01, -6.2066e-03,  9.9767e-01],
         [-1.0059e+00,  9.8958e-01, -4.8907e-04],
         [ 9.9702e-01, -1.8703e-03, -1.0003e+00],
         [ 9.9507e-01, -1.0030e+00, -6.6027e-03],


In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    # Concat positive and negative edge indices.
    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    # Label for positive edges: 1, for negative edges: 0.
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    # Note: The model is trained in a supervised manner using the given
    # `edge_label_index` and `edge_label` targets.
    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

# Train/Test Loop
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')


Epoch: 001, Loss: 0.6898, Val: 0.8125, Test: 0.6914
Epoch: 002, Loss: 0.6812, Val: 0.8750, Test: 0.6790
Epoch: 003, Loss: 0.6629, Val: 0.7500, Test: 0.6790
Epoch: 004, Loss: 0.6491, Val: 0.6875, Test: 0.6667
Epoch: 005, Loss: 0.6394, Val: 0.5000, Test: 0.6667
Epoch: 006, Loss: 0.6204, Val: 0.5000, Test: 0.6296
Epoch: 007, Loss: 0.5814, Val: 0.5000, Test: 0.5926
Epoch: 008, Loss: 0.6037, Val: 0.5000, Test: 0.5679
Epoch: 009, Loss: 0.5497, Val: 0.5000, Test: 0.5556
Epoch: 010, Loss: 0.5821, Val: 0.5000, Test: 0.5432
Epoch: 011, Loss: 0.6151, Val: 0.5625, Test: 0.5556
Epoch: 012, Loss: 0.6047, Val: 0.5625, Test: 0.5802
Epoch: 013, Loss: 0.5483, Val: 0.5625, Test: 0.5802
Epoch: 014, Loss: 0.5652, Val: 0.5625, Test: 0.5556
Epoch: 015, Loss: 0.5164, Val: 0.5625, Test: 0.5926
Epoch: 016, Loss: 0.5794, Val: 0.5625, Test: 0.5926
Epoch: 017, Loss: 0.5035, Val: 0.5625, Test: 0.6049
Epoch: 018, Loss: 0.5395, Val: 0.5000, Test: 0.6173
Epoch: 019, Loss: 0.5069, Val: 0.4375, Test: 0.6296
Epoch: 020, 

In [ ]:
z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)


In [ ]:
z.shape

torch.Size([31, 2])

In [ ]:
z

tensor([[ 0.2006,  0.5993],
        [ 1.0808,  0.3958],
        [ 1.7738,  0.2527],
        [-1.3969, -0.7702],
        [ 0.6044,  1.4926],
        [ 0.3546, -1.2677],
        [-1.0674,  0.7304],
        [ 0.4989,  1.3687],
        [-1.8316,  0.3944],
        [ 0.3855,  0.4681],
        [ 0.9900,  1.2023],
        [ 1.3051, -1.0147],
        [-0.6511,  1.3192],
        [-1.6022, -0.8433],
        [-0.6098,  0.2953],
        [-0.6706, -0.6197],
        [ 1.8986, -0.3647],
        [-0.1846,  1.1360],
        [-0.5961, -1.4654],
        [-0.0329, -0.6400],
        [ 1.0961,  0.8424],
        [ 1.3997, -1.0634],
        [ 0.3315,  1.4598],
        [-0.9722,  0.6993],
        [ 0.8069,  0.7934],
        [-1.6752, -0.4596],
        [-1.2179,  0.3345],
        [ 0.2456, -0.7085],
        [ 1.9910, -0.9511],
        [ 1.0780, -1.4862],
        [-0.3308, -1.2393]], grad_fn=<AddBackward0>)

In [ ]:
final_edge_index

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,
          6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,
          7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,
          8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,
          9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
         11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
         12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14,
         14, 14, 14, 14, 14,

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Assuming 'z' contains your node embeddings
node_embeddings = z.cpu().numpy()  # Convert to NumPy array if it's a PyTorch tensor

# Apply t-SNE to reduce dimensionality
tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
embeddings_2d = tsne.fit_transform(node_embeddings)

# Create a scatter plot for visualization
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], marker='o', s=20)
plt.title('t-SNE Visualization of Node Embeddings')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.show()


RuntimeError: ignored